# LLM Class

> 
- skip_showdoc: true
- skip_exec: true

## With design pattern

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
import uuid
import json
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

# ---------------- Adapter Pattern (LangChain LLM Wrapper) ----------------
class LLMAdapter(ABC):
    @abstractmethod
    def generate_response(self, messages: list) -> str:
        pass

class LangChainOpenAIAdapter(LLMAdapter):
    def __init__(self, model="gpt-4o"):
        self.llm = ChatOpenAI(
            model=model,
            temperature=0.7,
            api_key=os.getenv("openai_api_key")
        )

    def generate_response(self, messages: list) -> str:
        return self.llm.invoke(messages).content.strip()

# ---------------- Prompt Strategy ----------------
class PromptStrategy(ABC):
    @abstractmethod
    def build_messages(self, user_input: str) -> list:
        pass

class QAStrategy(PromptStrategy):
    def build_messages(self, user_input: str) -> list:
        return [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content=f"Q: {user_input}\nA:")
        ]

class SummarizeStrategy(PromptStrategy):
    def build_messages(self, user_input: str) -> list:
        return [
            SystemMessage(content="You summarize input text."),
            HumanMessage(content=f"Summarize:\n{user_input}")
        ]

# ---------------- Chat History ----------------
class ChatHistory:
    def __init__(self):
        self.data = {}

    def log(self, session_id, role, message):
        self.data.setdefault(session_id, []).append({
            "timestamp": datetime.now().isoformat(),
            "role": role,
            "message": message
        })

    def show(self, session_id):
        for entry in self.data.get(session_id, []):
            print(f"{entry['timestamp']} [{entry['role']}]: {entry['message']}")

    def export(self, session_id):
        return json.dumps(self.data.get(session_id, []), indent=2)

# ---------------- LLM Client ----------------
class LLMClient:
    def __init__(self, adapter: LLMAdapter, strategy: PromptStrategy):
        self.adapter = adapter
        self.strategy = strategy
        self.history = ChatHistory()
        self.session_id = str(uuid.uuid4())

    def chat(self, user_input):
        messages = self.strategy.build_messages(user_input)
        self.history.log(self.session_id, "user", user_input)
        response = self.adapter.generate_response(messages)
        self.history.log(self.session_id, "assistant", response)
        return response

    def view_history(self):
        self.history.show(self.session_id)

    def export_history(self):
        return self.history.export(self.session_id)

# ---------------- Entry Point ----------------
adapter = LangChainOpenAIAdapter()
strategy = QAStrategy()  # or SummarizeStrategy()
bot = LLMClient(adapter, strategy)


In [ ]:

print("\n🧠 Chat Session Started — Type 'exit' to quit.\n")

while True:
    msg = input("👤 You: ")
    if msg.strip().lower() in {"exit", "quit"}:
        break
    response = bot.chat(msg)
    print(f"🤖 Bot: {response}")

In [ ]:
# View history in Jupyter
bot.view_history()

# Export as JSON
json_output = bot.export_history()
print(json_output)


2025-04-21T14:38:45.428523 [user]: how is the weather today
2025-04-21T14:38:46.342267 [assistant]: I'm sorry, but I don't have real-time data access to provide current weather updates. You can check the latest weather information through a weather app or website for your location.
2025-04-21T14:38:52.994441 [user]: what day is it?
2025-04-21T14:38:54.119662 [assistant]: I'm sorry, I can't provide real-time information or current dates. Please check your device's calendar or clock for the current day.
2025-04-21T14:39:10.178454 [user]: whats the difference between django and fastapi
2025-04-21T14:39:17.439090 [assistant]: Django and FastAPI are both web frameworks for Python, but they have different design philosophies and use cases. Here's a comparison of the two:

1. **Speed and Performance**:
   - **FastAPI**: As the name suggests, FastAPI is designed for high performance. It is built on top of Starlette and is asynchronous by nature, making it well-suited for handling lots of concu